<h1><b>Statistique en Bioinformatique : </b> TME2 </h1>
<br>
L’objectif de ce TME est:
<br>
<ul>
<li> objectif 1: comprendre les différences d'alignement global et local, </li>
<li> objectif 2: reconstruire une matrice de substitution du type Blosum, </li>
</ul>
<br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><b>Soumission</b></p>
<ul>
<li>Renomer le fichier TME2.ipynb pour SBAS_TME_2_NomEtudiant1_NomEtudiant2.ipynb </li>
<li>Soumettre sur moodle </li>
</ul>
</div>

Nom etudiant 1 : Enzo GARCIA
<br>
Nom etudiant 2 : Maher TANTOUCH
<br>

<b> Exercice 1 </b>: 

<b>A.</b> Récupérez la séquence de la protéine “metL Bifunctional aspartokinase/homoserine dehydrogenase 2” (P00562) et de la protéine “lysC Lysine-sensitive aspartokinase 3” (P08660) chez E. coli via le site <a href="http://www.uniprot.org/"> Uniprot </a> au format .fasta. Reproduisez, à l’aide du site <a href="https://www.ebi.ac.uk/Tools/psa/"> psa</a>, l’alignement global et local vu en TD. Observez la modification des résultats en changeant la matrice de substitution et/ou les pénalités de gap (“Gap_penalty” et “Extend_penalty”).

<b>Exercice 2</b>: Score alignement global, local <br>
Faire une fonction pour calculer : 1) la pourcentage de identité, 2) la pourcentage de similarité, 3) le score d’un alignement en utilisant la matrix blosum62. Le calcul de la similarité doit prendre en considération les acides amine ayant la même propriété physico chimique. Dans notre cas, tous les acides amine qui ont des valeurs supérieur à zéro dans la matrice BLOSUM62. Le score basé sur la matrice blosum doit aussi prendre en considération les deux pénalités de gaps, ouverture et extension. Teste vos fonctions en utilisant le fichier test.fasta


In [112]:
import numpy as np

# Initialisation des variables
aa = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']  # 20 acides aminés
q = 20  # Ajusté à 20 acides aminés
gap_open = -5  # Pénalité d'ouverture de gap
gap_ext = 0.5  # Pénalité d'extension de gap

# Fichiers
input_test_f = 'test.fasta'
input_blosum_f = 'BLOSUM62.txt'

# Lecture du fichier test.fasta
def read_fasta(input_f):
    """
    Input1 input_f: nom du fichier fasta
    Output1 arr_seqs: Liste des séquences
    REMARQUE : Nous ne gardons que les séquences, les identifiants ne sont pas nécessaires
    """
    arr_seqs = []
    with open(input_f, 'r') as in_f:
        seq = ""
        for line in in_f:
            if line.startswith(">"):  ##pour quand la ligne commence par '>', c'est un en-tête
                if seq:
                    arr_seqs.append(seq)  ##ajout de la seq actuelle
                seq = ""  # Réinitialiser la séquence
            else:
                seq += line.strip()  ##pour ajout de la seq (en supprimant les retours à la ligne)
        if seq:  ##pour ajout de la dernière seq après la boucle
            arr_seqs.append(seq)
    
    return arr_seqs

testAln = read_fasta(input_test_f)
print(testAln)
# Cela devrait imprimer ['ACCD', 'A-CD'] lors de l'utilisation du fichier toy

['EADINIAFYQRDHGDNSPFDGPNGILAHAFQPGQGIGGDAHFDAEETWT', 'MADILVVFARGAHGDFHAFDGKGG-LAHAFGPGSGIGGDAHFDEDEFWT']


In [114]:
# Lecture du fichier BLOSUM
def read_blosum(input_f):
    """
    Input1 input_f : nom du fichier blosum
    Output1: une matrice de dimension [q-1, q-1] représentant la matrice de substitution
    REMARQUE : Vous devez éliminer les colonnes/lignes B, Z et * du fichier BLOSUM
    """
    blosum = np.zeros((q, q))  ##on ajuste a une matrice 20x20
    aa_dict = {aa[i]: i for i in range(len(aa))}
    
    with open(input_f, 'r') as in_f:
        for line in in_f:
            if line.startswith("#") or line.startswith(" "):  ##en vue d'ignorer les lignes de commentaire ou vides
                continue
            parts = line.split()
            if len(parts) == 1:  ##on va egalament ignorer les lignes qui ne correspondent pas à un acide aminé
                continue
            row_aa = parts[0]
            if row_aa not in aa_dict:
                continue
            row_idx = aa_dict[row_aa]
            for col_idx, score in enumerate(parts[1:]):
                if col_idx < q:
                    blosum[row_idx, col_idx] = int(score)
    
    return blosum

matr_62 = read_blosum(input_blosum_f)

In [116]:
def indentite_calcul(seq1, seq2, gap_open, gap_ext, matr_62):
    """
    Input1 seq1 : première séquence
    Input2 seq2 : deuxième séquence
    Input3 gap_open : pénalité d'ouverture de gap
    Input4 gap_ext : pénalité d'extension de gap
    Input5 matr_62 : matrice de substitution de dimension [q-1, q-1]
    Output1 ident: identité des séquences
    Output2 similaire: similarité des séquences
    Output3 score: score d'alignement
    """
    
    m = len(seq1)
    n = len(seq2)
    
    ##on fait une matrice de scores (initialisation)
    score_matrix = np.zeros((m + 1, n + 1))
    for i in range(m + 1):
        score_matrix[i, 0] = gap_open + gap_ext * i
    for j in range(n + 1):
        score_matrix[0, j] = gap_open + gap_ext * j
    
    #o#on fait le remplissage de la matrice de scores (Programmation Dynamique)
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if seq1[i - 1] == '-' or seq2[j - 1] == '-':  ##gap present -> on n'utilise pas la matrice BLOSUM
                match_score = gap_open if seq1[i - 1] == '-' or seq2[j - 1] == '-' else 0
            else:
                match_score = matr_62[aa.index(seq1[i - 1]), aa.index(seq2[j - 1])]
            
            score_matrix[i, j] = max(
                score_matrix[i - 1, j - 1] + match_score,  ##diagonal (match/mismatch)
                score_matrix[i - 1, j] + gap_ext,  ##si gap dans seq2
                score_matrix[i, j - 1] + gap_ext   #"si gap dans seq1
            )
    
    ##retour en arriere pour calculer le score d'alignement
    score = score_matrix[m, n]
    
    
    ##calcul de l'identité et de la similarité (en ignorant les gaps)
    identical = 0
    similar = 0
    for i in range(m):
        if seq1[i] != '-' and seq2[i] != '-':  ##on va ignorer les gaps dans le calcul de l'identité et de la similarité
            if seq1[i] == seq2[i]:  ##condition d'identité
                identical += 1
            elif matr_62[aa.index(seq1[i]), aa.index(seq2[i])] > 0:  ##condition de similarité
                similar += 1
    
    identitee = identical / max(m, n)  ##pourcentage d'identité
    similarity = similar / max(m, n)  #"pourcentage de similarité
    
    return identitee, similarity, score

identitee, similarity, score = indentite_calcul(testAln[0], testAln[1], gap_open, gap_ext, matr_62)
print("identitee= ", identitee, " similarity= ", similarity, " score= ", score)

    

identitee=  0.6122448979591837  similarity=  0.061224489795918366  score=  200.0


<b>Exercice 3</b>: Matrice de substitution <br>
Faire une programme (plusieurs fonctions seront nécessaires) pour produire une matrix de substitution comme Blosum. Utiliser l’alignement du fichiers <b>blocks.dat</b> Vous pouvez tester vous fonctions sue le fichier <b>blockToy.dat</b>.


In [118]:
from itertools import combinations, combinations_with_replacement
from collections import Counter

input_block_f = 'blockToy.dat'


# Pour un test simple, on peut définir :
# q = 3
#aa = ['A', 'B', 'C']


# Lecture du fichier d'alignement
from io import StringIO
import sys

def readAlnFile(input_f):
    """
    Lecture du fichier d'alignement et extraction des séquences alignées.
    
    Input1 : input_f : Nom du fichier d'alignement
    Output1 : aln : Liste des séquences alignées
    Output2 : cols : Nombre de colonnes (L)
    """
    with open(input_f, 'r') as f:
        aln = []
        for line in f:
            line = line.strip()
            if line and not line.startswith('>'):
                aln.append(line)
        cols = len(aln[0])  ##pour nombre de colonnes qu'on suppose constant
    return aln, cols

aln, cols = readAlnFile(input_block_f)
#cols = 43  ##pour les vrais données
cols = 5  ##test avec un fichier toy
print("Nombre de colonnes :", cols)










Nombre de colonnes : 5


In [120]:
##pour calculer les fréquences des paires de résidus
def frequencies(aln, cols, aa):
    """
    Fonction pour calculer la fréquence des paires de résidus dans les alignements.
    
    Input1 : aln : Liste des séquences alignées
    Input2 : cols : Nombre de colonnes dans l'alignement (L)
    Input3 : aa : Alphabet des résidus
    Output1 : pairs_freq_dict : Dictionnaire contenant les fréquences des paires de résidus
    """
    ##pour créer toutes les combinaisons de résidus (y compris les paires de résidus identiques)
    pairs_freq_dict = {x: 0 for x in combinations_with_replacement(aa, 2)}
    aa.sort()

    ##pour chaque colonne dans l'alignement, compter les paires de résidus
    for col in range(cols):
        column_residues = [seq[col] for seq in aln]
        ##comptage des occurrences des paires de résidus (en triant les paires)
        for pair in combinations(column_residues, 2):  ##pour utiliser des combinations et éviter les répétitions
            sorted_pair = tuple(sorted(pair))  ##trier la paire pour garantir que l'ordre ne pose pas de problème
            pairs_freq_dict[sorted_pair] += 1
    
    return pairs_freq_dict

##exemple de sortie : {('A', 'A'): 3, ('A', 'B'): 4, ('A', 'C'): 5, ('B', 'B'): 5, ('B', 'C'): 10, ('C', 'C'): 3}
pairs_freq_dict = frequencies(aln, cols, ['A', 'B', 'C'])
print(pairs_freq_dict)



{('A', 'A'): 3, ('A', 'B'): 4, ('A', 'C'): 5, ('B', 'B'): 5, ('B', 'C'): 10, ('C', 'C'): 3}


In [122]:
##fonction pour calculer la somme totale T
def calcule_T(pairs_freq_dict):
    """
    Calcul de la somme totale T des fréquences des paires de résidus.
    
    Input1 : pairs_freq_dict : Dictionnaire contenant les fréquences des paires de résidus
    Output1 : T : La somme totale
    """
    T = sum(pairs_freq_dict.values())  # Somme des fréquences des paires
    return T

T = calcule_T(pairs_freq_dict)
print("Somme totale T :", T)



Somme totale T : 30


In [124]:
##fonction pour calculer les probabilités pi
def calcule_p_is(pairs_freq_dict, T):
    """
    Calcul de la probabilité de chaque résidu.
    
    Input1 : pairs_freq_dict : Dictionnaire contenant les fréquences des paires de résidus
    Input2 : T : La somme totale
    Output1 : pi : Liste des probabilités
    """
    ##pour calculer la probabilité de chaque résidu à partir des fréquences des paires
    aa_set = set([item[0] for item in pairs_freq_dict.keys()] + [item[1] for item in pairs_freq_dict.keys()])
    pi = {residue: 0 for residue in aa_set}
    
    for pair, freq in pairs_freq_dict.items():
        pi[pair[0]] += freq / T
        pi[pair[1]] += freq / T
    
    return pi

pi = calcule_p_is(pairs_freq_dict, T)
print("Probabilités pi :", pi)


Probabilités pi : {'C': 0.7, 'B': 0.8, 'A': 0.5}


In [126]:
##fonction pour calculer les scores Sij
def calcule_S_i_j(pairs_freq_dict, p, T):
    """
    Calcul des scores pour chaque paire de résidus (Sij).
    
    Input1 : pairs_freq_dict : Dictionnaire contenant les fréquences des paires de résidus
    Input2 : p : Liste des probabilités pour chaque résidu
    Input3 : T : La somme totale
    Output1 : s : Score pour chaque paire de résidus
    """
    s = {}
    for pair, freq in pairs_freq_dict.items():
        p_i = p[pair[0]]
        p_j = p[pair[1]]
        score = (freq / T) / (p_i * p_j)  # Calcul du score Sij
        s[pair] = score
    
    return s

Sij = calcule_S_i_j(pairs_freq_dict, pi, T)
print("Scores Sij :", Sij)



Scores Sij : {('A', 'A'): 0.4, ('A', 'B'): 0.3333333333333333, ('A', 'C'): 0.4761904761904762, ('B', 'B'): 0.2604166666666666, ('B', 'C'): 0.5952380952380952, ('C', 'C'): 0.20408163265306126}


In [128]:
##affichage de la matrice Blosum
##maintenant, chaque score Sij est une valeur flottante et nous devons l'afficher correctement dans un format de matrice.
aa_list = list(pi.keys())  # Récupère l'alphabet des résidus
Sij_c = [[Sij.get((aa1, aa2), 0) for aa2 in aa_list] for aa1 in aa_list]

In [130]:
##affichage de la matrice Blosum
print('Matrice Blosum générée à partir de la base de données Blocks:')
row_format = "{:>5}|" * (len(aa_list) + 1)
print(row_format.format("", *aa_list))
for aa, row in zip(aa_list, Sij_c):
    print(row_format.format(aa, *["{:0.2f}".format(x) for x in row]))


Matrice Blosum générée à partir de la base de données Blocks:
     |    C|    B|    A|
    C| 0.20| 0.00| 0.00|
    B| 0.60| 0.26| 0.00|
    A| 0.48| 0.33| 0.40|
